In [1]:
!ls ../input/

openidataset  word-embeddings-openi-new


In [2]:
!pip install torchxrayvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 13.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import re
import csv
import umap
import seaborn as sns
import os
from PIL import Image
import nltk
from nltk import bigrams
import itertools
import cv2
import collections
from pathlib import Path
from collections import Counter 
from random import sample
import math
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import sentence_bleu
import pickle
import random

import torch
import torchvision

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchxrayvision as xrv
import skimage.io


from transformers import BertModel
from transformers import BertTokenizer
from torchvision import transforms, models
from skimage import data, io, filters
from torch.utils.data import DataLoader, Dataset
import torch.optim.lr_scheduler as lr_scheduler


import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def get_img_dict(train_ids,val_ids,test_ids):
    
    n_train = len(train_ids)
    n_val = len(val_ids)
    n_test = len(test_ids)
    
    train_idx = np.arange(n_train)
    val_idx = np.arange(n_train,n_train+n_val)
    test_idx = np.arange(n_train+n_val,n_train+n_val+n_test)
    
    img_ids = train_ids + val_ids + test_ids
    
    idx2img = {}
    img2idx = {}
    
    for i,img in enumerate(img_ids):
        idx2img[i] = img
        img2idx[img] = i
    return img_ids,idx2img,img2idx,train_idx,val_idx,test_idx


def get_image_word_edges(train_img_ids,train_df):
        keywords = train_df['Keywords'].values
        edges = set()
        for i,img in enumerate(train_img_ids):
            for w in keywords[i].split(','):
                edges.add((img,w))
        return np.array(list(edges))
        
    
    


In [6]:
img_dir = '../input/openidataset/images/images/'
data_dir = '../input/word-embeddings-openi-new/'

In [7]:
with open(data_dir + 'openi_word_vocab.pkl', 'rb') as f:
    word_vocab = pickle.load(f)
    
print(len(word_vocab))

1512


In [8]:
train_df = pd.read_csv(data_dir + 'train_reports_with_keywords.csv')
val_df = pd.read_csv(data_dir + 'val_reports_with_keywords.csv')
test_df = pd.read_csv(data_dir + 'test_reports_with_keywords.csv')

In [9]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(2566, 14)
(366, 14)
(733, 14)


In [10]:
train_df.head()

,Image,Frontal,Lateral,Comparison,Indication,Findings,Impression,MESH,Problems,Target,Problem_List,Prepoc_Problems,Prepoc_Target,Keywords
0,CXR3403_IM-1647-1001;CXR3403_IM-1647-2001,CXR3403_IM-1647-1001,CXR3403_IM-1647-2001,None.,XXXX-year-old female. Left chest pain. Right r...,the cardiomediastinal silhouette is normal in ...,negative for acute abnormality,normal,normal,the cardiomediastinal silhouette is normal in ...,['normal'],normal,"cardiomediastinal,silhouette,normal,size,conto...","size,effusion,contour,pleural,abnormality,pneu..."
1,CXR810_IM-2343-1001;CXR810_IM-2343-2001,CXR810_IM-2343-1001,CXR810_IM-2343-2001,None.,chest pain,normal heart size mediastinal and aortic conto...,no evidence of active cardiopulmonary disease,Calcified Granuloma/scattered;Spine/degenerative,Calcified Granuloma;Spine,normal heart size mediastinal and aortic conto...,"['calcified granuloma', 'spine']",calcified granuloma;spine,"normal,heart,size,mediastinal,aortic,contour,n...","pulmonary,visible,cardiopulmonary,contour,aort..."
2,CXR2474_IM-1003-1001,CXR2474_IM-1003-1001,NaN,NaN,Shortness of breath.,patchy interstitial infiltrates have developed...,bibasilar patchy airspace disease with bilater...,Infiltrate/lung/lower lobe/bilateral/interstit...,Infiltrate;Costophrenic Angle;Aorta;Airspace D...,patchy interstitial infiltrates have developed...,"['infiltrate', 'costophrenic angle', 'aorta', ...",infiltrate;costophrenic angle;aorta;airspace d...,"patchy,interstitial,infiltrate,develop,low,lob...","bilateral,pulmonary,infiltrate,pleural,normal,..."
3,CXR632_IM-2213-1001;CXR632_IM-2213-1002,CXR632_IM-2213-1001,CXR632_IM-2213-1002,Chest x-XXXX and XXXX.,"XXXX-year-old male with shortness of breath, X...",cardiac and mediastinal silhouette are unremar...,no acute cardiopulmonary abnormality,normal,normal,cardiac and mediastinal silhouette are unremar...,['normal'],normal,"cardiac,mediastinal,silhouette,unremarkable,lu...","lung,cardiopulmonary,cardiac,effusion,abnormal..."
4,CXR3101_IM-1453-1001;CXR3101_IM-1453-3001,CXR3101_IM-1453-1001,CXR3101_IM-1453-3001,None.,Multiple myeloma. Bone marrow transplant.,the heart size and pulmonary vascularity appea...,no evidence of active disease,"Spine/degenerative;Stents/abdomen;Aorta, Thora...","Spine;Stents;Aorta, Thoracic;Calcified Granuloma",the heart size and pulmonary vascularity appea...,"['spine', 'stents', 'aorta thoracic', 'calcifi...",spine;stents;aorta thoracic;calcified granuloma,"heart,size,pulmonary,vascularity,appear,within...","limit,pulmonary,pleural,see,normal,change,gran..."


In [11]:
openi_train_img_ids = [names.split(';')[0] for names in train_df['Frontal'].values]
openi_val_img_ids = [names.split(';')[0] for names in val_df['Frontal'].values]
openi_test_img_ids = [names.split(';')[0] for names in test_df['Frontal'].values]
len(openi_train_img_ids)

2566

In [12]:
openi_image_word_edges = get_image_word_edges(openi_train_img_ids,train_df)

In [13]:
openi_image_word_edges[:10]

array([['CXR260_IM-1090-1001', 'advanced'],
       ['CXR2015_IM-0664-2001', 'suspicious'],
       ['CXR3673_IM-1828-1001', 'acute'],
       ['CXR178_IM-0509-2001', 'pleural'],
       ['CXR1346_IM-0224-4001', 'lung'],
       ['CXR721_IM-2282-1001', 'masse'],
       ['CXR2210_IM-0817-1001', 'cardiomediastinal'],
       ['CXR2645_IM-1131-2001', 'cardiomediastinal'],
       ['CXR2644_IM-1130-1001', 'finding'],
       ['CXR934_IM-2432-1001', 'bilateral']], dtype='<U25')

In [14]:
openi_img_ids,openi_id2img,openi_img2id,openi_train_idx,openi_val_idx,openi_test_idx = get_img_dict(openi_train_img_ids,
                                                                            openi_val_img_ids,
                                                                            openi_test_img_ids)

## Image Graph

In [15]:
def read_image(img_path,transform):
    img = skimage.io.imread(img_path)
    img = xrv.datasets.normalize(img, 255) 

    # Check that images are 2D arrays
    if len(img.shape) > 2:
        img = img[:, :, 0]
    if len(img.shape) < 2:
        print("error, dimension lower than 2 for image")

    # Add color channel
    img = img[None, :, :]
    img = transform(img)
    return img


class ResnetDataset(Dataset):
    def __init__(self,images,transform,img_dir):
        self.images = images
        self.data_dir = img_dir
        self.tfm = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(self.data_dir,self.images[idx]+'.png'))
        if(self.tfm):
            img = self.tfm(img)
        return img



class DensenetDataset(Dataset):
    def __init__(self,images,transform,img_dir):
        self.images = images
        self.data_dir = img_dir
        self.tfm = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = read_image(os.path.join(self.data_dir,self.images[idx]+'.png'),self.tfm)
        return torch.tensor(img,dtype=torch.float32)
    
    
class Resnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet152(pretrained=True)
        modules2 = list(self.model.children())[:-2]
        self.fix2 = nn.Sequential(*modules2)
        for p in self.model.parameters():
            p.requires_grad=False
        self.gap = nn.AdaptiveAvgPool2d((1,1))
    def forward(self, x):
        img_feats = self.fix2(x)
        img_feats = self.gap(img_feats).squeeze(3).squeeze(2)
        return img_feats
    
    
    
class Densenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = xrv.models.DenseNet(weights="densenet121-res224-all")
        modules2 = list(self.model.children())[:-2]
        self.fix2 = nn.Sequential(*modules2)
        for p in self.model.parameters():
            p.requires_grad=False
        self.gap = nn.AdaptiveAvgPool2d((1,1))
    def forward(self, x):
        img_feats = self.fix2(x)
        img_feats = self.gap(img_feats).squeeze(3).squeeze(2)
        return img_feats

In [16]:

# dataset = DensenetDataset(openi_img_ids,transform,img_dir)
# img = dataset.__getitem__(0)
# print(img.shape)
# plt.imshow(img.squeeze(0),cmap='gray')

In [17]:
# model = Resnet()
model = Densenet()
model = model.to(device)

If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O /root/.torchxrayvision/models_data/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]


In [18]:
transform = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                xrv.datasets.XRayResizer(224)])

# transform = transforms.Compose([transforms.Resize((224,224)),
#                             transforms.ToTensor()])

openi_img_dataset = DensenetDataset(openi_img_ids,transform,img_dir)
openi_img_dataloader =  DataLoader(openi_img_dataset, batch_size = 8, shuffle=False)

Setting XRayResizer engine to cv2 could increase performance.


In [19]:
openi_img_feats = []
for img in openi_img_dataloader:
    model.eval()
    img = img.to(device)
    img_feat = model(img)
    openi_img_feats.append(img_feat)
openi_img_feats = torch.cat(openi_img_feats).detach().cpu().numpy()

print(openi_img_feats.shape)

(3665, 1024)


In [20]:
np.save('openi_img_feats.npy', openi_img_feats) # save

In [21]:
with open('openi_id2img.pkl', 'wb') as f:
    pickle.dump(openi_id2img, f)

with open('openi_img2id.pkl', 'wb') as f:
    pickle.dump(openi_img2id, f)
    
with open('openi_img_ids', 'wb') as f:
    pickle.dump(openi_img_ids, f)

with open('openi_train_img_ids', 'wb') as f:
    pickle.dump(openi_train_img_ids, f)
    
with open('openi_val_img_ids', 'wb') as f:
    pickle.dump(openi_val_img_ids, f)

with open('openi_test_img_ids', 'wb') as f:
    pickle.dump(openi_test_img_ids, f)

In [22]:
np.save('openi_train_img_idx.npy', openi_train_idx)
np.save('openi_val_img_idx.npy', openi_val_idx)
np.save('openi_test_img_idx.npy', openi_test_idx)

np.save('openi_image_word_edges.npy',openi_image_word_edges)